
# Introduction

This annotated jupyter notebook is intended to provide training material for users of a system for seasonal hydrological forecast in the Shire River Basin - a climate service developed under funding from FOCUS Africa project. 

The notebook focuses on one of the processes involved in this sytem, namely the process of calibration of a probabilistic (an initial condition ensemble) seasonal forecast to the reference historical data.

While it is intended as a training material, it assumes that the "trainee", or a user of this notebook has a basic understanding of:
- seasonal forecasting concepts such as seasonal foreasting with dynamical models and in particular understanding of an initial condition ensemble forecast
- statistical concepts such as probability, distributions, quantiles etc.
- data processing using python


test1

<img src="https://web.csag.uct.ac.za/~wolski/focus_africa/training_material/forecast_malawi/figures/forecast_fig2.png" />

test2

<img src="figures/forecast_fig2.png" alt="framework2" class="bg-primary mb-1" width="500px" />

test3

![test](figures/forecast_fig2.png)

test 4

![](forecast_fig2.png)


## A bit of a background
Hydrological modelling for water resources in Malawi is done at Malawi DCCMS using a suite of three models:
- rainfall-runoff model of catchment upstream from Lake Malawi
- Lake Malawi water balance model
- rainfall-runoff and river routing model for the Lower Shire River basin.

The hydrological models were developed/calibrated with a bespoke gridded, daily rainfall and air temperature dataset that are a blend of CRU and observational data. This dataset covers only the historical period of 1981-2018. 

This hydrological modelling suite does not currently operate in a seasonal forecast mode, and thus there is scope for development and implementation of seasonal hydrological forecast where these three models are routinely run with forcing data from a seasonal climate forecast.

The overall framework of the seasonal forecasting system is briefly described below, but the intention of this notebook is to present the processing of climate forecast data from a probabilistic climate forecast generated by a dynamical global climate model to the format suitable for forcing of the DCCMS hydrological model suite.  


## Adopted approach to implementing the DCCMS hydrological models in seasonal forecasting mode 
Overall framework for the operational seasonal hydrological forecast in the Lake Malawi Basin is illustrated in Fig. 1. In that, the hydrological forecast is issued every month and covers 6 monthly period starting on that month.
<center>
<img src="https://web.csag.uct.ac.za/~wolski/focus_africa/training_material/forecast_malawi/figures/forecast_fig1.png" alt="framework1" class="bg-primary mb-1" width="500px">
</center>
<center>
<i>Fig. 1 Timing of hydrological model simulations in the seasonal hydrological forecasting system</i>
</center>

The hydrological forecast is based on forecast climate variables generated by an ensemble of dynamical models. The entire ensemble can be processed, or alternatively only selected models. Since the models generate a probabilistic ensemble forecast, even using one model will involve multiple hydrological simulations. The system requires the following simulations with the suite of hydrological models (Fig. 2):
- Simulations to establish initial conditions for the forecast simulations. These simulations are based on a historical, observed time series of rainfall that spans from the beginning of record till the month prior to the date of issuing the forecast.
- Simulations forced by forecast data. These are multiple runs of each model forced by an ensemble of time series of climatic variables derived from the raw forecast data through a statistical calibration procedure.

Output of the hydrological simulations constitutes an ensemble of possible future hydrological states, and that is interpreted within a probabilistic framework - i.e. in terms of probabilities of exceedance etc.



<center>
<img src="./figures/forecast_fig2.png" alt="framework2" class="bg-primary mb-1" width="500px">
</center>
<center>
<i>Fig. 2 Schematic illustration of the outputs of the seasonal hydrological forecasting system</i>
</center>

## Datastream underlying the hydrological forecasting system

Fig. 3 illustrates the "flow" of data in the hydrological forecasting system. There are a number of steps involved:
- download of climate forecast data
- pre-processing (or homogeneization) of climate forecast data
- integration of monitoring/observational data with historical hydrological model forcing data
- downscaling and calibration of climate forecast data
- running of the three hydrological models with monitoring/observational data and seasonal forecast data

<b>This notebook describes only the process of forecast downscaling/calibration!</b>

<center>
<img src="https://web.csag.uct.ac.za/~wolski/focus_africa/training_material/forecast_malawi/figures/forecast_fig3.png" alt="framework3" class="bg-primary mb-1" width="500px">
</center>
<center>
<i>Fig. 3 Datastream underlying the hydrological forecasting system</i>
</center>


# Process of forecast downscaling & calibration

<center>
<img src="https://web.csag.uct.ac.za/~wolski/focus_africa/training_material/forecast_malawi/figures/forecast_fig4.png" alt="framework4" class="bg-primary mb-1" width="500px">
</center>
<center>
<i>Fig. 4 Forecast downcaling and calibration</i>
</center>

## Monthly data
Calibration of the monthly data using the IoV approach against observed monthly data (hydrological model input aggregated to monthly totals) (Johnson and Bowler, 2009).
The IoV approach is suited to ensure the condition of ensemble consistency (as described above, i.e. that the observations are statistically indistinguishable from a member of the forecast ensemble). That condition translates into two requirements 1) that the MSE of the ensemble mean is equal to climatological mean of ensemble variance (i.e. mean of variances of ensemble members of all forecasts of a given month at a given lead time), and 2) that climatological variance of an ensemble member is equal to the climatological variance of observations. This is achived by a set of linear transformation of the ensemble data as per Johnson and Bowler (2009). The calibration is implemented at each grid point, for each target month at each forecast lead time. 

## Daily data
Bias correction at daily time scale is carried out using a modified quantile-quantile mapping approach. The modification accounts for the requirement that daily bias-corrected data should yield monthly total corresponding to the calibrated monthly value. In order to achieve that - an iterative procedure of adjustment of quantiles is implemented (Fig. 5). This allows for achieving agreement between daily and monthly totals, and adherence of the daily data to the observational distribution. 

<center>
<img src="https://web.csag.uct.ac.za/~wolski/focus_africa/training_material/forecast_malawi/figures/forecast_fig5.png" alt="framework5" class="bg-primary mb-1" width="500px">
</center>
<center>
<i>Fig. 5 Bias-correcting of daily data</i>
</center>



# Prerequisites for the process implemented in this notebook

We do not implement here the entire process of generation of the seasonal hydrological forecast as illustrated in Fig. 3. Rather, we focus only on the Downscaling/Calibration step.

In order to implement this step - a number of pre-requisites are needed, namely:
- raw forecast data have to be downloaded from the source
- the raw data have to be harmonized to be compatible with this script. This involves:
    - individual forecasts have to be stored in separate netcdf files
    - the netcdf data file has to have a particular structure:
        - forecast data are stored in a four-dimensional array, with the following dimensions order: time,member,latitude,longitude
        - variable names have to be harmonized - this script uses <i>latitude</i> and <i>longitude</i> for latitude and longitude, <i>time</i> for time variable, <i>pr</i> for precipitation and <i>member</i> for ensemble member variable
        - the time variable encodes date/time of the period for which forecast is issued. This is feasible only if data from an individual forecast, i.e. a forecast issued on a particular date, are stored in an individual file.
    - forecast data have to be regrided to the same domain and spatial grid size as the observed reference data
    - directory structure and file naming convention has to be aligned with those adopted here (described below)
    - we consider that the forecast spans 6 months from the issuance date
    - for forecast - at least daily data have to be available. Monthly data are required, but it can be calculated from daily, if not pre-calculated during harmonization
    

## Data model
The sample data provided with this notebook are created under a particular "data model", i.e. a schematic that governs the directory structure and file naming conventions adopted to store the data. 

That data model requires some explanation before we proceed.


### Directory structure:

The data are stored in the following tree of directories:

<i>\[data_dir\]/\[ensemble\]/\[model\]/\[basetime\]/\[domain\]/\[variable\]</i>



<b><i>data_dir</i></b>

This is the directory in which harmonized forecast data are stored. That directory is specific to the HPC on which the system operates. This allows easy adaptation of the code if the system is ported or set-up on a different HPC, as well as setting up this notebook to work with sample data provided here. 

<b><i>ensemble, model</i></b>

These define the sub-directory structure, so that data for a particular model sourced from a particular ensemble are stored in a particular directory, e.g. data for the ECMWF's SEAS5.1 will be stored in:

<i>\[root_dir\]/ECMWF/SEAS51/</i>

<b><i>basetime</i></b>

Basetime denotes the temporary resolution of data. Data at monthly basetime, or resolution are stored in:

<i>\[root_dir\]/ECMWF/SEAS51/mon/</i>

while data at daily resolution are stored in:

<i>\[root_dir\]/ECMWF/SEAS51/day/</i>

<b><i>domain</i></b>

Domain defines a region over which data are processed, which allows for creating separate datastreams for different regions. In our case, data are stored in:

<i>\[root_dir\]/ECMWF/SEAS51/mon/malawi/</i>

<b><i>variable</i></b>

This level of directory structure allows for storing data for various variables separately. In our case we process only precipitation data, so we will store them in: 

<i>\[root_dir\]/ECMWF/SEAS51/mon/malawi/pr/</i>

### File naming convention:

We adopt the following file naming convention:

<i>\[var\]_\[basetime\]_\[ensemble\]_\[model\]_\[datatype\]_\[year\]\[month\].nc</i>

so data for monthly rainfall forecast issued in December 2023 will be stored in:

<i>pr_mon_ECMWF_SEAS51_fcst_202312.nc</i>

of course placed in an appropriate directory.


# Finally, we can start coding!

## Preliminaries

Firstly, we have to load all the python libraries we going to use:

In [1]:
import xarray as xr
import numpy as np
import os, glob, sys, calendar
import scipy.stats as stat
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

Now, we can set up input and output data files. 
The code in this notebook processes a forecast from a single seasonal forecasting system, namely ECMWF's SEAS5.1. 
In order to make the code universal, i.e. for it to be able to be quickly adapted to process data from a different forecasting system, we introduce a number of variables that identify the processed dataset

In [107]:
# the ensemble variable is introduced to allow for differentiation 
# between different sources of forecast data. In our notation, ECMWF describes a multi-model 
# ensemble distributed through the Copernicus Data Store.
ensemble="ECMWF"

#this is code of the particular model from the ensemble. 
model="SEAS51"

# This is the number of the month from which the forecast will be processed. 
# Note that "normally" python indexes variables starting from 0, 
# but the number that is used to define a calendar month is indexed from 1, 
# i.e. January is the first month of the year, and corresponds to the value of 1  
fcstmonth=10

# This is the year from which the forecast will be processed.
fcstyear=2023

# This is variable that is going to be processed. 
# This notebook describes processing of rainfall only
variable="pr"

# This defines a domain, or a region over which data are processed
domain="malawi"

#finally, a variable that controls whether or not existing output files are overwritten
overwrite=True

# this is definition of trace rainfall, i.e. amount which will be treated as 0 in calculations
traceday=0.1
tracemon=5

# iteration parameters used during the iterative adjustement of daily data
#convcrit expresses fraction difference betweeen monthly total from adjusted daily data and that in calibrated monthly data
convcrit=0.01
maxiter=100

We will subsequently derive some "secondary" variables

In [29]:
# This is abbreviated name of the forecast month
fcstmonth_abbr=calendar.month_abbr[fcstmonth]

# This is a two-digit code for the forecast month. 
# It is introduced here because input files are named using numerical value of the month
# in a two-digit format, i.e. the file name for January will have 
# the month coded as 01 rather than 1 
fcstmonth_code=str(fcstmonth).zfill(2)

In [30]:
# This is the root directory in which calibrated forecast data from all ensembles and models are stored
harmonizedfcst_root="/terra/projects/focus-africa/fcst_malawi/data/seasonal"
harmonizedfcst_root="/terra/projects/focus-africa/fcst_malawi/data/seasonal"

# This is the root directory in which calibrated forecast data from all ensembles and models are stored
calibratedfcst_root="/terra/projects/focus-africa/fcst_malawi/data/seasonal-calibrated"

# These are directories that store data for a particular ensemble and model
harmonizedfcstday_dir="{}/{}/{}/day/{}/{}".format(harmonizedfcst_root,ensemble,model,domain,variable)
calibratedfcstday_dir="{}/{}/{}/day/{}/{}".format(calibratedfcst_root,ensemble,model,domain,variable)
harmonizedfcstmon_dir="{}/{}/{}/mon/{}/{}".format(harmonizedfcst_root,ensemble,model,domain,variable)
calibratedfcstmon_dir="{}/{}/{}/mon/{}/{}".format(calibratedfcst_root,ensemble,model,domain,variable)

# These are input files - monthly and daily
harmonizedfcstmon_file="{}/{}_mon_{}_{}_{}{}.nc".format(harmonizedfcstmon_dir,variable,ensemble,model,fcstyear,fcstmonth_code)
harmonizedfcstday_file="{}/{}_day_{}_{}_{}{}.nc".format(harmonizedfcstday_dir,variable,ensemble,model,fcstyear,fcstmonth_code)

# These are output files - monthly and daily
calibratedfcstmon_file="{}/{}_mon_{}_{}_fcst-iov_{}{}.nc".format(calibratedfcstmon_dir,variable,ensemble,model,fcstyear,fcstmonth_code)
calibratedfcstday_file="{}/{}_day_{}_{}_fcst-iov_{}{}.nc".format(calibratedfcstday_dir,variable,ensemble,model,fcstyear,fcstmonth_code)

# These are input reference data files
referencemon_file="../data/obs_malawi/{}_mon_obs.nc".format(variable)
referenceday_file="../data/obs_malawi/{}_day_obs.nc".format(variable)

Now we will perform check for output files and directories - perhaps they already exist, i.e. were calculated before, and there is nothing to do? Unless we want to overwrite the old files, that is...

In [126]:
print("Checking data availability for forecast for {} {}:\n".format(fcstmonth_abbr, fcstyear))

print("Daily output file: {}".format(calibratedfcstday_file))
if os.path.exists(calibratedfcstday_file):
    print("exists")
    if overwrite==False:
        print("but overwrite if False. Nothing to process. Exiting...\n")
        sys.exit()
    else:
        print("but overwrite if True. Processing...\n")
else:
    print("does not exist. Processing...\n")

print("Monthly output file: {}".format(calibratedfcstmon_file))
if os.path.exists(calibratedfcstmon_file):
    print("exists")
    if overwrite==False:
        print("but overwrite if False. Nothing to process. Exiting...\n")
        sys.exit()
    else:
        print("but overwrite if True. Processing...\n")
else:
    print("does not exist. Processing...\n")
    
if not os.path.exists(calibratedfcstday_dir):
    print("Output directory {} does not exist. It has to be created manually. Exiting...\n".format(calibratedfcstday_dir))
    sys.exit()

if not os.path.exists(calibratedfcstmon_dir):
    print("Output directory {} does not exist. It has to be created manually. Exiting...".format(calibratedfcstmon_dir))
    sys.exit()


Checking data availability for forecast for Oct 2023:

Daily output file: /terra/projects/focus-africa/fcst_malawi/data/seasonal-calibrated/ECMWF/SEAS51/day/malawi/pr/pr_day_ECMWF_SEAS51_fcst-iov_202310.nc
exists
but overwrite if True. Processing...

Monthly output file: /terra/projects/focus-africa/fcst_malawi/data/seasonal-calibrated/ECMWF/SEAS51/mon/malawi/pr/pr_mon_ECMWF_SEAS51_fcst-iov_202310.nc
exists
but overwrite if True. Processing...



Now we will perform check for all necessary input files and directories.
Note that downscaling/calibration process requires:
- historical observational reference data
- historical forecast data (retrospective forecast)
- current harmonized (uncalibrated) forecast data

All of these data has to be available at daily and monthly time step


In [128]:
print("Checking observational reference period files")

#reference files
if not os.path.exists(referencemon_file):
    print("Reference observational data file {} does not exist. exiting...\n".format(referencemon_file))
    sys.exit()
else:
    print("Reference observational data file {} exists. Proceeding...".format(referencemon_file))
if not os.path.exists(referenceday_file):
    print("Reference observational daily data file {} does not exist. exiting...\n".format(referenceday_file))
    sys.exit()
else:
    print("Reference observational daily data file {} exists. Proceeding...\n".format(referenceday_file))

    
#retrospective forecast files
print("Checking retrospective forecast files for {}".format(fcstmonth_abbr))

pattern="{}/{}_mon_{}_{}_*{}.nc".format(harmonizedfcstmon_dir,variable,ensemble,model,fcstmonth_code)
harmonizedfcstmon_files=glob.glob(pattern)
if len(harmonizedfcstmon_files)==0:
    print("There are no harmonized files matching pattern {}. exiting...".format(pattern))
    sys.exit()
else:
    print("Found harmonized monthly files for {} years".format(len(harmonizedfcstmon_files)))
    
pattern="{}/{}_day_{}_{}_*{}.nc".format(harmonizedfcstday_dir,variable,ensemble,model,fcstmonth_code)
harmonizedfcstday_files=glob.glob(pattern)
if len(harmonizedfcstday_files)==0:
    print("There are no harmonized files matching pattern {}. exiting...".format(pattern))
    sys.exit()
else:
    print("Found harmonized daily files for {} years\n".format(len(harmonizedfcstday_files)))

    
print("Checking current forecast files for {} {}".format(fcstmonth_abbr, fcstyear))

print("Monthly input file: {}".format(harmonizedfcstmon_file))
calc_harmonizedfcstmon=False
if os.path.exists(harmonizedfcstmon_file):
    print("exists. Proceeding...")
else:
    print("Is missing. It will be calculated from daily data\n")
    calc_harmonizedfcstmon=True
    
print("Daily input file: {}".format(harmonizedfcstday_file))
if os.path.exists(harmonizedfcstday_file):
    print("exists. Proceeding...")
else:
    print("missing. Calculations cannot proceed. Exiting")
    sys.exit()


Checking observational reference period files
Reference observational data file ../data/obs_malawi/pr_mon_obs.nc exists. Proceeding...
Reference observational daily data file ../data/obs_malawi/pr_day_obs.nc exists. Proceeding...

Checking retrospective forecast files for Oct
Found harmonized monthly files for 37 years
Found harmonized daily files for 37 years

Checking current forecast files for Oct 2023
Monthly input file: /terra/projects/focus-africa/fcst_malawi/data/seasonal/ECMWF/SEAS51/mon/malawi/pr/pr_mon_ECMWF_SEAS51_202310.nc
exists. Proceeding...
Daily input file: /terra/projects/focus-africa/fcst_malawi/data/seasonal/ECMWF/SEAS51/day/malawi/pr/pr_day_ECMWF_SEAS51_202310.nc
exists. Proceeding...


# Calculating monthly integration for current forecast at daily time step, if necessary

Here we will calculate monthly total from daily data. This is a relatively simple operation that uses the xarray resample function.

In [33]:
if calc_harmonizedfcstmon==True:
    # opening daily file
    ds_day=xr.open_dataset(harmonizedfcstday_file)
    #calculating monthly total
    ds_mon=ds_day.resample(time="M").sum()
    # adding unit attribute
    ds_mon[variable].attrs["unit"]="mm/month"
    # adding description
    ds_mon.attrs["description"]="derived by integration of daily data to monthly total"
    #saving to monthly netcdf file
    ds_mon.to_netcdf(harmonizedfcstmon_file)


# Downscaling/calibration at monthly time step

## Methodology

Calibration at monthly time scale is here carried out using an approach called Inflation of Variance. That approach is described by...



## Implementation

The IoV is implemented using two functions - calibrate_iov_single and calibrate_iov_loocv
"loocv" stands for leave one out cross-validation. That function calibrates data for given forecast  based on statistics derived from all other forecsts issued for that forecast's month.

In [36]:
def calibrate_iov_single(_fcst,_obs,_target):
    if np.sum(np.isnan(_fcst[:,0]))==0:
        _fcst = _fcst[~np.isnan(_obs),:]
        _obs = _obs[~np.isnan(_obs)]
        
        #corecting bias in mean of the forecast
        _fmean=np.mean(np.nanmean(_fcst,axis=1))
        _fcst=_fcst-_fmean
        _target=_target-_fmean
        _omean=_obs.mean()
        _obs=_obs-_omean
        
        _f_t=np.nanmean(_fcst,axis=-1).reshape(-1,1) # "ensemble mean" at time steps (T) (1a)
        _epsil_t=_fcst-_f_t #anomalies (T,M)
        _sigma2_t=(np.nanmean((_fcst-_f_t)**2, axis=-1)) #eq. 1b ensemble variance at time steps (T)
        _sigma2_x=np.var(_obs) #eq. 2a "climatological variance of true state" (1 value)
        _mu_f=np.mean(_f_t) #grand mean of the ensemble (1 value)
        _mu_x=np.mean(_obs) #mean of observations (1 value)
        _sigma2_f2b=np.mean((_f_t-_mu_f)**2) #eq. 2b - variance of ensemble mean (one value)
        _sigma2_f2c=np.mean(np.nanmean((_fcst-_mu_f)**2,axis=-1)) #2c - mean variance of ensemble #(one value)
        _sigma2_e=np.mean(_sigma2_t) #Eq. 2d average ensemble variance (one value)
 
        _rho=stat.pearsonr(_f_t.flatten(),_obs)[0] #they are adjusted earlier to cover the same time
        _alpha=abs(_rho)*(_sigma2_x**0.5)/(_sigma2_f2b**0.5) #eq. 7a 2b is the correct one!!!
        _beta2=(1-_rho**2)*_sigma2_x/_sigma2_e #eq. 7b
        
        #this is for target
        _f_t_target=np.nanmean(_target,axis=-1).reshape(-1,1)
        _epsil_t_target=_target-_f_t_target #anomalies (T,M)
        _target_calib=_alpha*_f_t_target+(_beta2**0.5)*_epsil_t_target
        _target_calib=_target_calib+_omean
        
        return(_target_calib)
    else:
        cont=True
        _output=np.copy(_target)
        _output[:]=np.nan
        return(_output)
    
def calibrate_iov_loocv(_fcst,_obs):
    if np.sum(np.isnan(_fcst[:,0]))==0:
        _output=np.copy(_fcst)
        _output[:]=np.nan
        for i in range(_fcst.shape[0]):
            _target_temp=_fcst[i,:]
            _fcst_temp=np.delete(_fcst,i,0)
            _obs_temp=np.delete(_obs,i,0)
            _output[i,:]=calibrate_iov_single(_fcst_temp,_obs_temp,_target_temp)
        return(_output)
    else:
        _output=np.copy(_fcst)
        _output[:]=np.nan        
        return(_output)  

### Reading monthly data

In [37]:
print("reading monthly reference data from: {}".format(referencemon_file))
refmon=xr.open_dataset(referencemon_file)[variable]

print("reading monthly retro forecast data")
fcstmon=xr.open_mfdataset(harmonizedfcstmon_files).load()[variable]


reading monthly reference data from: ../data/obs_malawi/pr_mon_obs.nc
reading monthly retro forecast data


### Calibrating

Code below loops through lead times, because calibration has to be done for a particular target month at a particular lead time
calibration is done using apply_ufunc which basically iterates though individual grid cells and implements the calibrate_loocv function on each grid cell
before the apply_ufunc is iplemented, some data processing is done:
- we find out the calendar month of the target month, i.e. month at give lead time
- we select data for that month from the reference dataset
- also, we select all data for that month from the retrospective forecasts. Mind that retrospective forecast were read only for the currently analysed issuance month. In this way, retro forecast data for target month are all at the same leadtime
- subsequently, we align the reference and reforecast data along the time axis. This is done by: 
    - creating an empty "placeholder" array for reference data of the same spatial and temporal extent as the reforecast data.
    - finding times when the actual reference data overlaps that "placeholder"
    - filling "placeholder" with data from the reference dataset


In [38]:
# since we are iterating through lead times, the individual lead time outputs are stored into a list. 
# That list will be converted to xarray once all is calculated
fcstmoncalib=[]

#iterating through leadtimes
for leadtime in range(6):
    #this will be calendar month of the target month at current lead time 
    calendarmon=fcstmon.time[leadtime].dt.month.data
    
    # selecting data for the target month only
    refmonselected=refmon.sel(time=refmon.time.dt.month==calendarmon)
    fcstmonselected=fcstmon.sel(time=fcstmon.time.dt.month==calendarmon)
    
    #creating a placeholder for reference data
    refmonarray=fcstmonselected[:,0,:,:].copy(deep=True)
    
    #filling it with nan
    refmonarray[:]=np.nan
    
    # finding dates in reference data for which reforecast is available
    overlap=[x for x in refmonselected.time.data if x in fcstmonselected.time.data]
    
    #this will have the same dimensions as fcstselected, and will be filled with reference data, when available, otherwise nan
    refmonoverlap=refmonselected.sel(time=overlap)
    
    #this corresponds to the coverage of forecast, but has data only where obs data are available
    refmonarray.loc[dict(time=refmonoverlap.time)]=refmonoverlap
    

    #calibrating
    print("calibrating at {} month leadtime".format(leadtime))
    temp=xr.apply_ufunc(
        calibrate_iov_loocv, 
        fcstmonselected.load(),
        refmonarray,
        input_core_dims=[["time","member"],["time"]],
        vectorize=True,
        output_core_dims=[["time","member"]]    
    )
    
    #some housekeeping
    #re-organizing dimensions
    temp=temp.transpose("time","member","latitude","longitude")
    
    #adding attributes so that the output dataset is CF compliant
    temp.longitude.attrs={
        "units":"degrees_east",
        "long_name":"longitude",
        "axis":"X",
        "standard_name":"longitude" 
    }
    temp.latitude.attrs={
        "units":"degrees_north",
        "long_name":"latitude",
        "axis":"Y",
        "standard_name":"latitude" 
    }
    #appending to the list that stores outputs
    fcstmoncalib.append(temp)
print("done")

calibrating at 0 month leadtime
calibrating at 1 month leadtime
calibrating at 2 month leadtime
calibrating at 3 month leadtime
calibrating at 4 month leadtime
calibrating at 5 month leadtime
done


### Finishing off the monthly process

In [39]:
# Concatenating outputs from individual lead times (so far stored in a list) into a single data array
fcstmoncalib=xr.concat(fcstmoncalib, dim="time")
# making sure there is a correct sequencing of dates
fcstmoncalib=fcstmoncalib.sortby("time")
# Since we are processing rainfall - negative values are not realistic, although they may emerge from the IoV calibration process
# this makes sure negative values are not retained in the calibrated monthly data
fcstmoncalibnoneg=fcstmoncalib.where(fcstmoncalib>0,0)
fcstmoncalib=fcstmoncalibnoneg.where(~np.isnan(fcstmoncalib))

Note that calibration is done on the entire available set of retro forecasts, including the current forecast, so the output array (fcstcalib) holds data for all years. Since we are interested only in the current forecast - we need to select only these data. It's a bit complicated, because we cannot just select by the nominal year and month of the current forecast...

In [40]:
#finding out the first day of the current forecast
fday=pd.to_datetime("{}-{}-{}".format(fcstyear,fcstmonth,1))
#finding out its last day
lday=fday+pd.DateOffset(months=7)
#selecting
fcstmoncalib_current=fcstmoncalib.sel(time=slice(fday.strftime("%Y-%m-%d"),lday.strftime("%Y-%m-%d")))

At last we can save the output to the file

In [41]:
#saving the selected data to output file
fcstmoncalib_current.to_netcdf(calibratedfcstmon_file)
print("written {}".format(calibratedfcstmon_file))


written /terra/projects/focus-africa/fcst_malawi/data/seasonal-calibrated/ECMWF/SEAS51/mon/malawi/pr/pr_mon_ECMWF_SEAS51_fcst-iov_202310.nc


# Bias-correcting daily data

Now that we have downscaled and calibrated data at monthly time scale, we can proceed with aligning daily data so that its monthly total aligns with calibrated monthly data.


## Methodology


## Implementation

Bias-correction of daily data is achieved by applying a bespoke bias-correction function over each grid point, separately for each month (lead time) of the forecast. This function implements the process described above and illustrated in Fig. 5. 

In [112]:
def qqm_monadjust(_fcstref,_obsref,_fcstday,_fcstmon,_convcrit,_maxiter):
    #making sure reference period does not have nans
    _fref=_fcstref[~np.isnan(_fcstref)]
    
    #fitting ecdf to data in reference period
    ecdf = ECDF(_fref.flatten())
    
    #preparing array to store the output
    _fcstdaybc=_fcstday.copy()
    _fcstdaybc[:]=np.nan

    #iterating through members of the forecast ensemble
    for m in range(_fcstday.shape[1]):
        #extracting data for current member
        _mfcstday=_fcstday[:,m]
        _mfcstmon=_fcstmon[:,m]
        
        #deriving quantiles for daily forecast data
        _mfcstday_quant=ecdf(_mfcstday.flatten())
        
        # finding rainfall values in observations that corresponds to given quantiles
        # this a temporary value that will be further adjusted through iterative process 
        _mfcstdaybc=np.nanquantile(_obsref.flatten(), _mfcstday_quant)
        # calculating correction factor, i.e. ratio of monthly total to sum of daily values
        # this is temporary value that will be adjusted through iterative process
        
        _corfacmon=_mfcstmon/_mfcstdaybc.sum() #this is in terms of rainfall values
        
        # for stability of iterations, i.e. to avoid divergence of the iterative process, 
        # we set up minimum and maximum possible value of... 
        _quantcorfac_min,_quantcorfac_max=0.01,4
        
        if _corfacmon>1:
            _quantcorfac_min=1
            _quantcorfac=_quantcorfac_min
        else:
            _quantcorfac_max=1
            _quantcorfac=_quantcorfac_max
            
        
        #this is a special case when calibrated monthly total is small. Daily data are reset to 0 then.
        if _mfcstmon<tracemon:
            _corfacmon=1
            _mfcstdaybc=_mfcstday.copy()
            _mfcstdaybc[:]=0
            
        # implementing iterative adjustement until convergence criterion is reached
        i=0
        while np.abs(_corfacmon-1)>_convcrit:
            i+=1
            #calculating adjusted quantiles
            _quant_adj=_mfcstday_quant*_quantcorfac
            
            #making sure adjusted quantiles are realistic
            _quant_adj[_quant_adj>1]=1
            
            # calculating rainfall values for the adjusted quantiles
            _mfcstdaybc=np.nanquantile(_obsref.flatten(), _quant_adj)
            
            # calculating adjustment needed for actual rainfall values
            _corfacmon=_mfcstmon/_mfcstdaybc.sum()
            
            if _corfacmon>1:
                # if corfacmon>1 then monthly total is too small, and daily values need to go up, 
                # for this we raise minimum possible value of quantile correcton factor
                _quantcorfac_min=_quantcorfac
            else:
                #if corfacmon<1 then monthly total is too large, and daily values need to go down 
                # for this we reduce maximum possible value of quantile correction factor
                _quantcorfac_max=_quantcorfac
                
            _quantcorfac=(_quantcorfac_min+_quantcorfac_max)/2
            
            if i==_maxiter:
                print("maxiter reached",_mfcstmon,_mfcstdaybc.sum(),_mfcstday.sum(),_corfacmon)
                break
        
        _fcstdaybc[:,m]=_mfcstdaybc
    return _fcstdaybc.reshape(*_fcstday.shape)

In [113]:
print("reading monthly reference data from: {}".format(referencemon_file))
referencedayds=xr.open_dataset(referenceday_file)
refday=referencedayds[variable]

print("opening all daily files for given month ({})".format(fcstmonth))
fcstday=xr.open_mfdataset(harmonizedfcstday_files).load()[variable]

print("opening monthly calibrated forecast for given month ({})".format(fcstmonth))
fcstmon=xr.open_dataset(calibratedfcstmon_file).load()[variable]


reading monthly reference data from: ../data/obs_malawi/pr_mon_obs.nc
opening all daily files for given month (10)
opening monthly calibrated forecast for given month (10)


In [114]:
print("bias correcting...")
maxiter=100
convcrit=0.01

fcstdaybc=[]
for leadtime in range(6):
    print("leadtime: {}".format(leadtime))
    
    calendarmon=fcstday.time[leadtime].dt.month.data
    
    # selecting data for the target month only
    refdayselected=refday.sel(time=refday.time.dt.month==calendarmon)
    fcstdayselected=fcstday.sel(time=fcstday.time.dt.month==calendarmon)
    
    #selecting data for this month only - all months
    #obsclim=obs.sel(time=obs.time.dt.month==tgtmonth)
    #fcstdayclim=fcstdayall.sel(time=fcstdayall.time.dt.month==tgtmonth)
    
    #creating a placeholder for reference data
#    refarray=fcstselected[:,0,:,:].copy(deep=True)
    
    #filling it with nan
#    refarray[:]=np.nan
    
    # finding dates in reference data for which reforecast is available
    overlap=[x for x in refdayselected.time.data if x in fcstdayselected.time.data]
    
    #this will have the same dimensions as fcstselected, and will be filled with reference data, when available, otherwise nan
    refdayoverlap=refdayselected.sel(time=overlap)
    
    #this corresponds to the coverage of forecast, but has data only where obs data are available
    #refarray.loc[dict(time=refsubset.time)]=refsubset

    
    
    #selecting overlap of obs and forecast
#    overlap=[x for x in obsclim.time.data if x in fcstdayclim.time.data]
#    obsoverlap=obsclim.sel(time=overlap)
#    (refsubset)

    overlap=[x for x in fcstdayselected.time.data if x in refdayselected.time.data]
#    overlap=[x for x in fcstdayclim.time.data if x in obsclim.time.data]
#    fcstoverlap=fcstdayclim.sel(time=overlap)
    fcstdayoverlap=fcstdayselected.sel(time=overlap)

    
    fcstdaytarget=fcstday.sel(time="{}-{}".format(fcstyear,fcstmonth_code))
    fcstmontarget=fcstmon.sel(time="{}-{}".format(fcstyear,fcstmonth_code))
        
    #replacing values less than trace with random numbers. This is done in order to allow adjustement of the number of rain days.
    rdata=np.copy(refdayoverlap)
    rdata[:]=np.random.uniform(trace/10,trace,len(rdata.flatten())).reshape(*rdata.shape)
    refdayoverlap=refdayoverlap.where((refdayoverlap<trace)==False, rdata)
    del rdata

    rdata=np.copy(fcstdayoverlap)
    rdata[:]=np.random.uniform(trace/10,trace,len(rdata.flatten())).reshape(*rdata.shape)
    fcstdayoverlap=fcstdayoverlap.where((fcstdayoverlap<trace)==False, rdata)
    del rdata

    rdata=np.copy(fcstdaytarget)
    rdata[:]=np.random.uniform(trace/10,trace,len(rdata.flatten())).reshape(*rdata.shape)
    fcstdaytgt=fcstdaytarget.where((fcstdaytarget<trace)==False, rdata)
    del rdata

    temp=xr.apply_ufunc(
        qqm_monadjust, 
        fcstdayoverlap.load(),
        refdayoverlap,
        fcstdaytarget.load().rename({"time":"time1"}),
        fcstmontarget.rename({"time":"time2"}),
        convcrit,
        maxiter,
        input_core_dims=[["time","member"],["time"],["time1","member"],["time2","member"],[],[]],
        vectorize=True,
        output_core_dims=[["time1","member"]]
    )

    fcstdaytargetbc=temp.rename({"time1":"time"}).transpose("time","member","latitude","longitude")
    fcstdaybc.append(fcstdaytargetbc)

bias correcting...
leadtime: 0
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...
leadtime: 1
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...
leadtime: 2
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...
leadtime: 3
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...
leadtime: 4
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...
leadtime: 5
(1023, 51, 20, 9) (1023, 20, 9) (31, 51, 20, 9) (1, 51, 20, 9)
bias correcting...


### Finishing off the daily process

In [119]:
fcstdaybc=np.round(fcstdaybc,2)
#fcstdaybc=xr.concat(fcstdaybc,"time")

#Checking for unrealistic daily values
maxval=np.max(fcstdaybc.data)
print("daily maximum {}".format(maxval))

if maxval>350:
    print("unrealistic max {}".format(maxval))

#saving the selected data to output file
fcstdaybc.to_netcdf(calibratedfcstday_file)
print("written {}".format(calibratedfcstday_file))



daily maximum 48.79999923706055
written /terra/projects/focus-africa/fcst_malawi/data/seasonal-calibrated/ECMWF/SEAS51/day/malawi/pr/pr_day_ECMWF_SEAS51_fcst-iov_202310.nc
